## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Summarization

Text summarization involves condensing lengthy textual content into a brief format while retaining its essential information and significance. The primary aim is to extract key details from a text document and present them concisely and comprehensibly. It plays a crucial role across various domains, including healthcare, aiding in efficient communication and decision-making.


- **Model**: `en.summarize.clinical_guidelines_large.pipeline`
- **Model Description**: This pretrained pipeline is built on top of the `summarizer_clinical_guidelines_large` model, which is based on Flan-T5-large. This model is fine-tuned to summarize clinical guidelines, currently focusing on Asthma and Breast Cancer, into four different sections: Overview, Causes, Symptoms, and Treatments. The context length of this model is 768 tokens.

In [5]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [6]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [7]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [8]:
model_name = "en-summarize-clinical-guidelines-large-pipeline"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

### A. Create an endpoint

In [9]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [10]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial setup

In [11]:
docs = [

    """Medical Specialty: Ophthalmology
    Description: A 67-year-old male patient presents with complaints of blurry vision and difficulty reading. (Medical Transcription Sample Report)


    SUBJECTIVE: This 67-year-old male patient presents with complaints of blurry vision and difficulty reading. He mentions that these symptoms have been gradually worsening over the past year.


    OBJECTIVE: Visual acuity was tested and measured at 20/50 in both eyes. Intraocular pressure was within normal range. Fundoscopic examination revealed mild age-related macular degeneration.


    ASSESSMENT: Age-related macular degeneration.


    PLAN:


    1- Prescribed a specific brand of nutritional supplements known to slow the progression of macular degeneration.


    2- Scheduled a follow-up appointment in three months to monitor the patient's condition and assess any changes in visual acuity.""",


    """Medical Specialty: Psychiatry
    Description: A 29-year-old female patient presents with complaints of persistent sadness and loss of interest. (Medical Transcription Sample Report)


    SUBJECTIVE: This 29-year-old female patient presents with complaints of persistent sadness, loss of interest in activities, and decreased energy. She reports experiencing these symptoms for the past six months.


    OBJECTIVE: The patient appeared depressed during the examination. No abnormalities were noted in vital signs or physical examination.


    ASSESSMENT: Major depressive disorder.


    PLAN:


    1- Initiated treatment with an antidepressant medication, starting at a low dose and planning for gradual titration as needed.


    2- Recommended psychotherapy sessions, such as cognitive-behavioral therapy, to address the patient's depressive symptoms.


    3- Scheduled a follow-up appointment in four weeks to assess treatment response and make any necessary adjustments."""
]



sample_text = """Medical Specialty: Dermatology
Description: A 35-year-old female patient presented with a complaint of skin rash. (Medical Transcription Sample Report)


SUBJECTIVE: This 35-year-old female patient reports having itchy rashes on her body for the past few weeks. She mentions changing her cleaning products and recently started using a new laundry detergent.


OBJECTIVE: The patient's skin was examined and showed multiple red, raised, and pruritic lesions on various areas of the body. No signs of infection or scaling were noted.


ASSESSMENT: Allergic contact dermatitis.


PLAN:


1- Prescribed topical corticosteroid cream to alleviate itching and inflammation. Advised the patient to avoid contact with the suspected irritant.


2- Follow-up appointment scheduled in two weeks to assess improvement and adjust treatment if necessary."""


### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [12]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data)

,summary
0,"Overview of the disease: The patient has allergic contact dermatitis, which is a condition where the skin is itchy and red due to exposure to an irritant. The patient mentions changing her cleaning products and recently starting using a new laundry detergent. Causes: The suspected irritant is a new laundry detergent. Symptom: The patient has itchy rashes on her body for the past few weeks. Treatment recommendations: The patient is prescribed topical corticosteroid cream to alleviate itching and inflammation. The patient is advised to avoid contact with the suspected irritant. Follow-up appointment scheduled in two weeks to assess improvement and adjust treatment if necessary."


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [13]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data)

,summary
0,"Overview of the disease: The patient has age-related macular degeneration, which is a condition where the eye's vision becomes blurry and difficult to read. Causes: The cause of age-related macular degeneration is unknown, but it is believed to be related to a combination of genetic and environmental factors. Symptom: The patient has blurry vision and difficulty reading. Treatment recommendations: The patient was prescribed a specific brand of nutritional supplements known to slow the progression of macular degeneration. A follow-up appointment was scheduled in three months to monitor the patient's condition and assess any changes in visual acuity."
1,"Overview of the disease: The patient is experiencing major depressive disorder, which is a type of mood disorder that affects the entire person's life. The patient has persistent sadness, loss of interest in activities, and decreased energy. Causes: The patient's major depressive disorder is caused by a combination of genetic and environmental factors. Symptom: The patient has persistent sadness, loss of interest in activities, and decreased energy. Treatment recommendations: The patient is being treated with antidepressant medication and recommended psychotherapy sessions to address the patient's depressive symptoms. The patient is scheduled for a follow-up appointment in four weeks to assess treatment response and make any necessary adjustments."


### JSON Lines


In [14]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [15]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"summary": "Overview of the disease: The patient has age-related macular degeneration, which is a condition where the eye's vision becomes blurry and difficult to read. Causes: The cause of age-related macular degeneration is unknown, but it is believed to be related to a combination of genetic and environmental factors. Symptom: The patient has blurry vision and difficulty reading. Treatment recommendations: The patient was prescribed a specific brand of nutritional supplements known to slow the progression of macular degeneration. A follow-up appointment was scheduled in three months to monitor the patient's condition and assess any changes in visual acuity."}
{"summary": "Overview of the disease: The patient is experiencing major depressive disorder, which is a type of mood disorder that affects the entire person's life. The patient has persistent sadness, loss of interest in activities, and decreased energy. Causes: The patient's major depressive disorder is caused by a combinatio

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [16]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [17]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )



In [18]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [20]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [21]:
process_s3_json_output_and_save(validation_json_file_name)

,summary
0,"Overview of the disease: The patient has age-related macular degeneration, which is a condition where the eye's vision becomes blurry and difficult to read. Causes: The cause of age-related macular degeneration is unknown, but it is believed to be related to a combination of genetic and environmental factors. Symptom: The patient has blurry vision and difficulty reading. Treatment recommendations: The patient was prescribed a specific brand of nutritional supplements known to slow the progression of macular degeneration. A follow-up appointment was scheduled in three months to monitor the patient's condition and assess any changes in visual acuity."
1,"Overview of the disease: The patient is experiencing major depressive disorder, which is a type of mood disorder that affects the entire person's life. The patient has persistent sadness, loss of interest in activities, and decreased energy. Causes: The patient's major depressive disorder is caused by a combination of genetic and environmental factors. Symptom: The patient has persistent sadness, loss of interest in activities, and decreased energy. Treatment recommendations: The patient is being treated with antidepressant medication and recommended psychotherapy sessions to address the patient's depressive symptoms. The patient is scheduled for a follow-up appointment in four weeks to assess treatment response and make any necessary adjustments."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [23]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [24]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"summary": "Overview of the disease: The patient has age-related macular degeneration, which is a condition where the eye's vision becomes blurry and difficult to read. Causes: The cause of age-related macular degeneration is unknown, but it is believed to be related to a combination of genetic and environmental factors. Symptom: The patient has blurry vision and difficulty reading. Treatment recommendations: The patient was prescribed a specific brand of nutritional supplements known to slow the progression of macular degeneration. A follow-up appointment was scheduled in three months to monitor the patient's condition and assess any changes in visual acuity."}
{"summary": "Overview of the disease: The patient is experiencing major depressive disorder, which is a type of mood disorder that affects the entire person's life. The patient has persistent sadness, loss of interest in activities, and decreased energy. Causes: The patient's major depressive disorder is caused by a combinatio

In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-summarize-clinical-guidelines-large--2024-05-08-08-10-19-681


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

